## Save and Restore
- Save the paramters of a model to disk
- Restore the paramters of a model from the checkpoint
- Customized model initialization
- How to finetune a model



### Save the paramters of your model to disk.

In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
tf.reset_default_graph()

In [2]:
# model define
def prelu(x, name = 'prelu'):
    with tf.variable_scope(name):
        alphas = tf.get_variable('alpha', x.get_shape()[-1], initializer=tf.constant_initializer(0.25), regularizer = l2_regularizer, dtype = tf.float32)
    pos = tf.nn.relu(x)
    neg = tf.multiply(alphas,(x - abs(x)) * 0.5)
    return pos + neg

def first_conv(input, num_output, name):
  with tf.variable_scope(name):
    zero_init = tf.zeros_initializer()
    network = tf.layers.conv2d(input, num_output, kernel_size = [3, 3], strides = (2, 2), padding = 'same', kernel_initializer = xavier, bias_initializer = zero_init, kernel_regularizer = l2_regularizer, bias_regularizer = l2_regularizer)
    network = prelu(network, name = name)
    return network

def block(input, name, num_output):
    with tf.variable_scope(name):
        network = tf.layers.conv2d(input, num_output, kernel_size = [3, 3], strides = [1, 1], padding = 'same', kernel_initializer = tf.random_normal_initializer(stddev=0.01), use_bias = False , kernel_regularizer = l2_regularizer)
        network = prelu(network, name = 'name'+ '1')
        network = tf.layers.conv2d(network, num_output, kernel_size = [3, 3], strides = [1, 1], padding = 'same', kernel_initializer = tf.random_normal_initializer(stddev=0.01), use_bias = False, kernel_regularizer = l2_regularizer)
        network = prelu(network, name = 'name'+ '2')
        network = tf.add(input, network)
        return network
l2_regularizer= tf.contrib.layers.l2_regularizer(1.0)
xavier = tf.contrib.layers.xavier_initializer_conv2d() 
def get_shape(tensor):
    static_shape = tensor.shape.as_list()
    dynamic_shape = tf.unstack(tf.shape(tensor))
    dims = [s[1] if s[0] is None else s[0] for s in zip(static_shape,dynamic_shape)]
    return dims
def infer(input,embedding_size=512):
    with tf.variable_scope('conv1_'):
        network = first_conv(input, 64, name = 'conv1')
        network = block(network, 'conv1_23', 64)
    with tf.variable_scope('conv2_'):
        network = first_conv(network, 128, name = 'conv2')
        network = block(network, 'conv2_23', 128)
        network = block(network, 'conv2_45', 128)
    with tf.variable_scope('conv3_'):
        network = first_conv(network, 256, name = 'conv3')
        network = block(network, 'conv3_23', 256)
        network = block(network, 'conv3_45', 256)
        network = block(network, 'conv3_67', 256)
        network = block(network, 'conv3_89', 256)
    with tf.variable_scope('conv4_'):
        network = first_conv(network, 512, name = 'conv4')
        network = block(network, 'conv4_23', 512)
    with tf.variable_scope('feature'):
        #BATCH_SIZE = network.get_shape()[0]
        dims = get_shape(network)
        print(dims)
        #BATCH_SIZE = tf.shape(network)[0]
        #feature = tf.layers.dense(tf.reshape(network,[BATCH_SIZE, -1]), 512, kernel_regularizer = l2_regularizer, kernel_initializer = xavier)
        feature = tf.layers.dense(tf.reshape(network,[dims[0], np.prod(dims[1:])]), embedding_size, kernel_regularizer = l2_regularizer, kernel_initializer = xavier)
    return feature

In [95]:
# inference
tf.reset_default_graph()
#image = tf.random_normal([1,112,96,3])
#image = tf.constant(np.random.normal(size=[1,112,96,3]),dtype=tf.float32)
img_init = np.random.normal(size=[1,112,96,3])
image = tf.constant(img_init,dtype=tf.float32)
with tf.variable_scope('sphere_net'):
    feature = infer(image)

[1, 7, 6, 512]


In [96]:
vars = tf.global_variables()
print vars[0]

<tf.Variable 'sphere_net/conv1_/conv1/conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>


In [97]:
# loss
pred = tf.layers.dense(feature,1,name='pred')
print pred.get_shape()
loss = tf.nn.l2_loss(pred-1)
optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

(1, 1)


In [98]:
# train and save model
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in xrange(500):
        loss_np,pred_np,_ = sess.run([loss,pred,optimizer])
        if i % 100 ==0:
            print loss_np,pred_np      
            saver.save(sess,'model/sphere',global_step=i,write_meta_graph=False)
            pred_nd = sess.run(pred)
            print(pred_nd)
      

0.45344 [[ 0.04769798]]
[[ 0.06438893]]
0.0117716 [[ 0.84656197]]
[[ 0.84942561]]
0.000257565 [[ 0.9773035]]
[[ 0.97773653]]
5.47655e-06 [[ 0.99669045]]
[[ 0.99675322]]
1.71822e-07 [[ 0.99941379]]
[[ 0.99942195]]


In [99]:
# restore the model and test
tf.reset_default_graph()
ckpt = tf.train.get_checkpoint_state('model')

print type(ckpt)
print ckpt.model_checkpoint_path
image = tf.constant(img_init,dtype=tf.float32)
print image
#image = tf.random_normal([1,112,96,3])
#image = tf.constant(np.random.normal(size=[1,112,96,3]),dtype=tf.float32)
with tf.variable_scope('sphere_net'):
    feature = infer(image)
pred = tf.layers.dense(feature,1,name='pred')
saver = tf.train.Saver(tf.trainable_variables())

with tf.Session() as sess:
    saver.restore(sess,ckpt.model_checkpoint_path)
    #saver.restore(sess,'model/sphere-100')
    print(sess.run(pred))

<class 'tensorflow.python.training.checkpoint_state_pb2.CheckpointState'>
model/sphere-400
Tensor("Const:0", shape=(1, 112, 96, 3), dtype=float32)
[1, 7, 6, 512]
INFO:tensorflow:Restoring parameters from model/sphere-400
[[ 0.99942195]]


## Customized model initialization

In [106]:
tf.reset_default_graph()
from tensorflow.python.training import checkpoint_utils
image = tf.constant(img_init,dtype=tf.float32)
with tf.variable_scope('sphere_net_left'):
    feature_left = infer(image)
with tf.variable_scope('sphere_net_right'):
    feature_right = infer(image)
pred = tf.layers.dense(feature_left,1,name='pred')
dense_var = [var for var in tf.trainable_variables() if 'pred' in var.name]
print dense_var
saver = tf.train.Saver(tf.trainable_variables())
checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_left/'})
checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_right/'})
checkpoint_utils.init_from_checkpoint('model/sphere-100',{'pred/':'pred/'})

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    #saver.restore(sess,ckpt.model_checkpoint_path)
    #saver.restore(sess,'model/sphere-100')
    print(sess.run(pred))


[1, 7, 6, 512]
[1, 7, 6, 512]
[<tf.Variable 'pred/kernel:0' shape=(512, 1) dtype=float32_ref>, <tf.Variable 'pred/bias:0' shape=(1,) dtype=float32_ref>]
INFO:tensorflow:Initialize variable sphere_net_left/conv3_/conv3_67/conv2d_1/kernel from checkpoint model/sphere-100 with sphere_net/conv3_/conv3_67/conv2d_1/kernel
INFO:tensorflow:Initialize variable sphere_net_left/conv1_/conv1/conv1/alpha from checkpoint model/sphere-100 with sphere_net/conv1_/conv1/conv1/alpha
INFO:tensorflow:Initialize variable sphere_net_left/conv1_/conv1_23/conv2d_1/kernel from checkpoint model/sphere-100 with sphere_net/conv1_/conv1_23/conv2d_1/kernel
INFO:tensorflow:Initialize variable sphere_net_left/feature/dense/kernel from checkpoint model/sphere-100 with sphere_net/feature/dense/kernel
INFO:tensorflow:Initialize variable sphere_net_left/conv3_/conv3/conv2d/bias from checkpoint model/sphere-100 with sphere_net/conv3_/conv3/conv2d/bias
INFO:tensorflow:Initialize variable sphere_net_left/conv3_/conv3_89/name

INFO:tensorflow:Initialize variable sphere_net_right/conv3_/conv3_45/name1/alpha from checkpoint model/sphere-100 with sphere_net/conv3_/conv3_45/name1/alpha
INFO:tensorflow:Initialize variable sphere_net_right/conv3_/conv3_45/conv2d_1/kernel from checkpoint model/sphere-100 with sphere_net/conv3_/conv3_45/conv2d_1/kernel
INFO:tensorflow:Initialize variable sphere_net_right/conv3_/conv3/conv2d/kernel from checkpoint model/sphere-100 with sphere_net/conv3_/conv3/conv2d/kernel
INFO:tensorflow:Initialize variable sphere_net_right/conv3_/conv3_67/name1/alpha from checkpoint model/sphere-100 with sphere_net/conv3_/conv3_67/name1/alpha
INFO:tensorflow:Initialize variable sphere_net_right/conv3_/conv3_89/conv2d/kernel from checkpoint model/sphere-100 with sphere_net/conv3_/conv3_89/conv2d/kernel
INFO:tensorflow:Initialize variable sphere_net_right/conv4_/conv4_23/name2/alpha from checkpoint model/sphere-100 with sphere_net/conv4_/conv4_23/name2/alpha
INFO:tensorflow:Initialize variable sphere

## Finetune your model

In [115]:
tf.reset_default_graph()
from tensorflow.python.training import checkpoint_utils
image = tf.constant(img_init,dtype=tf.float32)
with tf.variable_scope('sphere_net'):
    feature_left = infer(image)
#with tf.variable_scope('sphere_net_right'):
#    feature_right = infer(image)
pred = tf.layers.dense(feature_left,1,name='pred')
loss = tf.nn.l2_loss(pred-1)
#optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
optimizer = tf.train.GradientDescentOptimizer(0.001)
dense_var = [var for var in tf.trainable_variables() if 'pred' in var.name]
grads = optimizer.compute_gradients(loss,dense_var)
train_op = optimizer.apply_gradients(grads)

print dense_var
saver_vars = [var for var in tf.trainable_variables() if 'pred' not in var.name]
#saver = tf.train.Saver(saver_vars)
saver = tf.train.Saver(tf.trainable_variables())
#checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_left/'})
#checkpoint_utils.init_from_checkpoint('model/sphere-100',{'sphere_net/':'sphere_net_right/'})
#checkpoint_utils.init_from_checkpoint('model/sphere-100',{'pred/':'pred/'})

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    #saver.restore(sess,ckpt.model_checkpoint_path)
    saver.restore(sess,'model/sphere-100')
    for i in xrange(500):
        loss_np,pred_np,_ = sess.run([loss,pred,train_op])
        if i % 100 ==0:
            print loss_np,pred_np      
            #saver.save(sess,'model/sphere',global_step=i,write_meta_graph=False)
            pred_nd = sess.run(pred)
            print(pred_nd)
      

[1, 7, 6, 512]
[<tf.Variable 'pred/kernel:0' shape=(512, 1) dtype=float32_ref>, <tf.Variable 'pred/bias:0' shape=(1,) dtype=float32_ref>]
INFO:tensorflow:Restoring parameters from model/sphere-100
0.0113363 [[ 0.84942561]]
[[ 0.8499887]]
0.00535868 [[ 0.89647532]]
[[ 0.89686245]]
0.00253303 [[ 0.92882377]]
[[ 0.92908984]]
0.00119736 [[ 0.95106411]]
[[ 0.95124698]]
0.000565992 [[ 0.96635503]]
[[ 0.96648079]]
